In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!cp -avr /content/drive/'My Drive'/DetectingSarcasm .

'/content/drive/My Drive/DetectingSarcasm' -> './DetectingSarcasm'
'/content/drive/My Drive/DetectingSarcasm/Models' -> './DetectingSarcasm/Models'
'/content/drive/My Drive/DetectingSarcasm/Models/LSTM2DMaxPool.py' -> './DetectingSarcasm/Models/LSTM2DMaxPool.py'
'/content/drive/My Drive/DetectingSarcasm/Models/AttantionLSTM.py' -> './DetectingSarcasm/Models/AttantionLSTM.py'
'/content/drive/My Drive/DetectingSarcasm/Models/BidirectionalLSTM.py' -> './DetectingSarcasm/Models/BidirectionalLSTM.py'
'/content/drive/My Drive/DetectingSarcasm/Models/LSTM.py' -> './DetectingSarcasm/Models/LSTM.py'
'/content/drive/My Drive/DetectingSarcasm/Models/Weights' -> './DetectingSarcasm/Models/Weights'
'/content/drive/My Drive/DetectingSarcasm/Notebooks' -> './DetectingSarcasm/Notebooks'
'/content/drive/My Drive/DetectingSarcasm/Notebooks/LSTMs.ipynb' -> './DetectingSarcasm/Notebooks/LSTMs.ipynb'
'/content/drive/My Drive/DetectingSarcasm/Notebooks/BERT.ipynb' -> './DetectingSarcasm/Notebooks/BERT.ipynb

In [3]:
%cd DetectingSarcasm/Notebooks/Experements

/content/DetectingSarcasm/Notebooks/Experements


In [4]:
!ls

CleanedDataLSTMs.ipynb	TuneLSTMs.ipynb


In [0]:
import sys
sys.path.append('../../Models')
# sys.path.append('../')

In [0]:
import torch
from torchtext import data

import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as F

import spacy

import time
import copy
from torch.optim import lr_scheduler

from torchtext.vocab import Vectors, GloVe
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import re

In [0]:
df = pd.read_json("../../Data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df = df.drop(['article_link'], axis=1)

We will use 20% of dataset as test set.

In [0]:
X = df['headline']
y = df['is_sarcastic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Loading test and train sets to csv:

In [0]:
!mkdir torchtext_data

In [0]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv("torchtext_data/train.csv", index=False)
test_data.to_csv("torchtext_data/test.csv", index=False)

Prepearing data:

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
fix_length = 24
TEXT = data.Field(sequential=True, tokenize="spacy", fix_length=fix_length)
LABEL = data.LabelField(dtype=torch.long, sequential=False)

In [0]:
train_data, test_data = data.TabularDataset.splits(
    path="torchtext_data/", train="train.csv", 
    test="test.csv",format="csv", skip_header=True, 
    fields=[('Text', TEXT), ('Label', LABEL)]
)

In [0]:
batch_size = 16

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), sort_key=lambda x: len(x.Text),
    batch_size=batch_size,
    device=device)

Making vocabluary with GloVe with dimantion of 300:

In [16]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 399207/400000 [00:36<00:00, 11212.17it/s]

In [0]:
word_embeddings = TEXT.vocab.vectors
output_size = 2 #two classes
num_layers = 1
hidden_size = 128 #num of units in our NN
embedding_length = 300 #dim of GloVe vector
vocab_size = len(TEXT.vocab)

Defining the model:


*   Simple LSTM - LSTMClassifier class 
*   LSTM with Attanrion - AttantionLSTMClassifier class 
*   Bidirectional LSTM - BidirectionalLSTMClassifier class
*   Bidirectional LSTM with 2D MaxPool layer - LSTM2DMaxPoolClassifier class





In [0]:
from LSTM2DMaxPool import *

In [0]:
model = LSTM2DMaxPoolClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, num_layers, word_embeddings)

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.000174282, weight_decay=4.62355e-05)
criterion = nn.CrossEntropyLoss()

In [21]:
model = model.to(device)
criterion = criterion.to(device)

100%|█████████▉| 399207/400000 [00:50<00:00, 11212.17it/s]

In [0]:
dataiter_dict = {'train': train_iterator, 'val': test_iterator}
dataset_sizes = {'train':len(train_data), 'val':len(test_data)}

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 200

    val_loss = []
    train_loss = []
    val_acc = []
    train_acc = []

    uncorrect_texts = []
    correct_labels = np.array([])

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            sentiment_corrects = 0
            tp = 0.0 # true positive
            tn = 0.0 # true negative
            fp = 0.0 # false positive
            fn = 0.0 # false negative

            # Iterate over data.
            for batch in dataiter_dict[phase]:
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    text = batch.Text
                    label = batch.Label
                    label = torch.autograd.Variable(label).long()

                    if torch.cuda.is_available():
                      text = text.cuda()
                      label = label.cuda()
                    if (batch.Text.size()[1] is not batch_size):
                      continue
                    
                    outputs = model(text)
                    outputs = F.softmax(outputs,dim=-1)                   
                    loss = criterion(outputs, label)

                    if phase == 'val' and epoch == num_epochs - 1:
                      itos = np.array(TEXT.vocab.itos) #getting indexes to words
                      mask = torch.max(outputs, 1)[1] != label #finding places where model made mistake
                      uncorrect_idxs = np.nonzero(mask).reshape(-1) #finding corresponding indexes
                      uncorrect_text = itos[text[:,uncorrect_idxs].cpu()].T #getting corresponding text
                      uncorrect_texts.append(uncorrect_text)
                      labels = np.array(label[mask].cpu()) #finding true label, where model made mistakes
                      correct_labels = np.append(correct_labels, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * text.size(0)
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == label)

                tp += torch.sum(torch.max(outputs, 1)[1] & label)
                tn += torch.sum(1-torch.max(outputs, 1)[1] & 1-label)
                fp += torch.sum(torch.max(outputs, 1)[1] & 1-label)
                fn += torch.sum(1-torch.max(outputs, 1)[1] & label)
                
            epoch_loss = running_loss / dataset_sizes[phase]
           
            sentiment_acc = float(sentiment_corrects) / dataset_sizes[phase]

            if phase == 'train':
                train_acc.append(sentiment_acc)
                train_loss.append(epoch_loss)
            elif phase == 'val':
                val_acc.append(sentiment_acc)
                val_loss.append(epoch_loss)

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                name = str(type(model))
                torch.save(model.state_dict(), '../../Models/Weights/'+ name[name.index('.')+1:-2] +'_model_test.pth')

            if phase == 'val' and epoch == num_epochs - 1:
                recall = tp / (tp + fn)
                print('recall {:.4f}'.format(recall))

        print()

    confusion_matrix = [[int(tp), int(fp)],[int(fn), int(tn)]]
    precision = tp / (tp + fp)
    f1 = 2*(precision*recall)/(precision+recall)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(float(best_loss)))

    results = {'time': time_elapsed, 
               'recall': recall,
               'precision': precision,
               'f1': f1, 
               'conf_matr': confusion_matrix,
               'val_loss': val_loss, 
               'train_loss': train_loss, 
               'val_acc': val_acc, 
               'train_acc': train_acc}

    mistakes = np.array(uncorrect_texts)
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, results, mistakes, correct_labels.astype(int)

Train model:

In [27]:
_, _, mistakes, correct_labels = train_model(model, criterion, optimizer, scheduler = None, num_epochs=20)

starting
Epoch 1/20
----------
train total loss: 0.7518 
train sentiment_acc: 0.8044
val total loss: 0.7592 
val sentiment_acc: 0.7954
saving with loss of 0.7591504617057256 improved over previous 200

Epoch 2/20
----------
train total loss: 0.7196 
train sentiment_acc: 0.8253
val total loss: 0.7259 
val sentiment_acc: 0.8160
saving with loss of 0.7258594654641062 improved over previous 0.7591504617057256

Epoch 3/20
----------
train total loss: 0.6952 
train sentiment_acc: 0.8421
val total loss: 0.7192 
val sentiment_acc: 0.8251
saving with loss of 0.7192030641517799 improved over previous 0.7258594654641062

Epoch 4/20
----------
train total loss: 0.6988 
train sentiment_acc: 0.8412
val total loss: 0.7088 
val sentiment_acc: 0.8323
saving with loss of 0.7087544359001223 improved over previous 0.7192030641517799

Epoch 5/20
----------
train total loss: 0.6706 
train sentiment_acc: 0.8619
val total loss: 0.6961 
val sentiment_acc: 0.8401
saving with loss of 0.6960579120637985 improved 

In [0]:
mistakes_list = []
for batch in mistakes:
  for headline in batch:
    mistakes_list.append(headline)

In [0]:
detokenized_text = [] 
for i in range(len(mistakes_list)):
    headline = ' '.join(mistakes_list[i]) 
    headline = re.sub(' <pad>|<unk>', '', headline)
    detokenized_text.append(headline) 

Some examples, where model made mistake:

In [36]:
df = pd.DataFrame({'Mistakes of the model': detokenized_text, 'Real Label': correct_labels.tolist()})
df.head(20)

,Mistakes of the model,Real Label
0,hamilton,0
1,receiving thanks,0
2,permission denied,0
3,naked leadership,0
4,april cruelty,0
5,quantum lip,0
6,doing time,0
7,robin hood foundation,0
8,refugee blues,0
9,family business,0
